# Neural Collaborative Filtering

Las redes neuronales son un subconjunto de algoritmos dentro de la familia del *machine learning* que ofrecen unos excelentes resultados a la mayoría de los problemas de aprendizaje supervisado. A diferencia de los modelos de *machine learning* clásicos, las redes neuronales suelen requerir poca preparación de los datos por parte de los científicos de datos, ya que los propios modelos se encargan de aprender dichos pre-procesamientos. Las redes neuronales han revolucionado el *machine learning* siendo capaces de mejorar la capacidad predictiva de la mayoría de los modelos tradicionales.

El campo de los sistemas de recomendación en general y del filtrado colaborativo en particular no ha sido ajeno a esta revolución y se han desarrollado algoritmos de filtrado colaborativo utilizando redes neuronales. A esta familia de algoritmos se la conoce como *Neural Collaborative Filtering*.

## Generalized Matrix Factorization (GMF)

El primer hito por alcanzar de los modelos de redes neuronales para filtrado colaborativo era lograr manejar la dispersión de la matriz de votaciones durante el entrenamiento de la red. La idea más básica que se nos podría ocurrir cuando construimos una red neuronal para resolver el problema de filtrado colaborativo es la de pasarle el vector de votaciones del usuario. Sin embargo, esta idea es inviable al ser este vector disperso. ¿Qué entrada le damos a la red cuando el usuario no ha votado el ítem? Todas las posibles respuestas reportan complicaciones:

- Poner un 0 en los votos faltantes indica a la red que al usuario no le ha gustado el ítem.
- Poner un valor alto en los votos faltantes indica a la red que al usuario le encanta el ítem.
- Poner la votación media general, la votación media del usuario o la votación media del ítem, no permitirá a la red aprender las características propias del usuario al tener una dispersión muy alta de los votos (>90%).

Por tanto, la resolución de este problema implica definir una estrategia de entrada para la red que sí permita manejar la dispersión de los datos. En concreto, necesitamos alimentar a la red con duplas `<usuario, item>` en las que la salida sea una `votación`. Generalmente, los usuarios y los ítems son representados por un código numérico (ej. el usuario `4` o el ítem `42`), pero ¿qué sucede si la entrada de la red son estos códigos numéricos? Que la red tendrá que corregir durante su aprendizaje el sesgo introducido por el valor numérico de los usuarios y los ítems: la red deberá aprender que el usuario `1000` no es 1000 veces más grande que el usuario `1`, si no que tienen esos números por puro azar.

Es por esto por lo que, en lugar de utilizar los códigos de los usuarios e ítems como entrada se recurre a utilizar una capa de *embedding*. La capa de *embedding* es un tipo de capa densa especial en la que se garantiza que todas sus entradas son `0` menos `1`, es decir, se representa una entrada discreta mediante *one-hot-encoding*. En el caso del filtrado colaborativo, se definen dos capas de *embedding* como entrada, una para los usuarios y otra para los ítems, que tienen por dimesión máxima el número de usuarios y el número de ítems respectivamente. Cuando se quiere predecir la votación del usuario `4` para el ítem `42`, las neuronas del *embedding* correspondientes a dicho usuario e ítem se pondrán a `1` y el resto quedarán a `0`.

A partir de este punto, se plantean diferentes arquitecturas de red que permiten estimar el voto que un usuario le otorgará a un ítem. La más sencilla de todas ellas se denomina *Generalized Matrix Factorization (GMF)* y tiene el siguiente aspecto:

![GMF](https://i.ibb.co/3hNBVK3/gmf.png)

Tras las capas de *embedding* se coloca una capa *dot* que combina las dos entradas mediante el producto escalar. La salida de esta capa es el voto. Hay que destacar que esta arquitectura es equivalente al modelo PMF, puesto el tamaño de la capa *dot* define el número de factores latentes a combinar linealmente y los pesos de la red equivalen a $p_u$ y $q_i$ de dicho modelo.

Veamos como implementar este modelo con `keras`.

Importamos las librerías necesarias:

In [ ]:
import urllib.request

import math
import numpy as np

from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense, Concatenate, Dot

Definimos el número de usuarios e ítems:

In [ ]:
NUM_USERS = 943
NUM_ITEMS = 1682

Cargamos los votos de entrenamiento. La carga de estos datos, por imposición de `keras` no se hace en una matriz como en los modelos de factorización matricial. Se generan dos *arrays* con los códigos de los usuarios y los ítems y un tercer *array* con las votaciones:

In [ ]:
X_train = [np.array([], dtype=int), np.array([], dtype=int)]
y_train = np.array([], dtype=int)

training_file = urllib.request.urlopen("https://drive.upm.es/s/tDdluElfGInyUnU/download")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  X_train[0] = np.append(X_train[0], int(u))
  X_train[1] = np.append(X_train[1], int(i))
  y_train = np.append(y_train, int(rating))

In [ ]:
X_train

In [ ]:
y_train

Cargamos también los votos de test del mismo modo:

In [ ]:
X_test = [np.array([], dtype=int), np.array([], dtype=int)]
y_test = np.array([], dtype=int)

test_file = urllib.request.urlopen("https://drive.upm.es/s/Jn75Vg6okOPsgZu/download")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  X_test[0] = np.append(X_test[0], int(u))
  X_test[1] = np.append(X_test[1], int(i))
  y_test = np.append(y_test, int(rating))

In [ ]:
X_test

In [ ]:
y_test

Los hiper-parámetros de nuestro modelo serán el número de factores latentes (`latent_dim`) y el número de iteraciones del entrenamiento (`epochs`).

In [ ]:
latent_dim = 5
epochs = 10

Definimos nuestra arquitectura:

In [ ]:
user_input = Input(shape=[1])
user_embedding = # defina la capa de embedding del usuario
user_vec = Flatten()(user_embedding)

item_input = Input(shape=[1])
item_embedding = # defina la capa de embedding del item
item_vec = Flatten()(item_embedding)

output = # defina la dot que combine user_vec e item_vec

GMF = Model([user_input, item_input], output)

Compilamos y entrenamos el modelo:

In [ ]:
GMF.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
GMF.summary()
GMF.fit(X_train, y_train, epochs=epochs, verbose=1)

Estimamos las predicciones de test:

In [ ]:
y_pred = GMF.predict(X_test)
y_pred

Medimos el error:

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

## Multi Layer Perceptron (MLP)

El modelo GMF tiene un problema: no aporta nada nuevo. Los usuarios e ítems son mapeados en un espacio latente y el voto es calculado como una combinación lineal de los factores latentes de cada usuario e ítem. Este modelo utiliza redes neuronales, pero no explota su principal arma: romper la linealidad de las operaciones.

Para ello, el modelo MLP mejora al modelo GMF permitiendo una combinación no-lineal de los factores latentes. Su arquitectura es la siguiente:

![MLP](https://i.ibb.co/pRpD4Hv/mlp.png)

Al igual que en GMF, los usuarios e ítems son representados mediante dos capas de *embedding*, sin embargo, estas no son combinadas mediante una capa *dot* si no que se unen en una capa *concatenate* a la que le suceden diferentes capas densas. A esa sucesión de capas densas se les conoce como MLP y su topología se configurar en función del conjunto de datos. Finalmente, se da como salida la predicción del voto.

Veamos su implementación en `keras`.

Definimos los mismos hiper-parámetros que en el modelo anterior:

In [ ]:
latent_dim = 5
epochs = 10

Definimos la arquitectura, en este caso con dos capas densas para el MLP de 20 y 10 neuronas:

In [ ]:
user_input = Input(shape=[1])
user_embedding = # defina la capa embedding del usuario
user_vec = Flatten()(user_embedding)

item_input = Input(shape=[1])
item_embedding = # defina la capa embedding del item
item_vec = Flatten()(item_embedding)

concat = # defina la capa concatenate para combinar user_vec e item_vec

output = # defina las capas necesarias para su MLP

MLP = Model([user_input, item_input], output)

Compilamos y entrenamos el modelo:

In [ ]:
MLP.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
MLP.summary()
MLP.fit(X_train, y_train, epochs=epochs, verbose=1)

Calculamos las predicciones:

In [ ]:
y_pred = MLP.predict(X_test)
y_pred

Medimos el error:

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

## Referencias

He, X., Liao, L., Zhang, H., Nie, L., Hu, X., & Chua, T. S. (2017, April). **Neural collaborative filtering**. In Proceedings of the 26th international conference on world wide web (pp. 173-182).
